In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import os
from copy import deepcopy
import random 
import cv2

In [3]:
dt = {}
for i in range(6):
    for j in range(6):
        dt[str(i)+str(j)] = str

#load dataframe
df1 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\train_faces.csv", dtype = dt)
df2 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\train_landmarks.csv", dtype= dt)

df3 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\sample_prediction.csv", dtype= dt)

df1.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,hwJUKjziH0.jpg,22,20,14,4,43,40,35,53,25,...,31,41,13,32,33,10,2,30,50,44
1,smzyIVQ4Ve.jpg,3,50,43,14,5,13,33,52,1,...,41,35,45,30,44,22,51,0,32,15
2,IT3H2QbaJY.jpg,55,25,43,10,33,20,22,13,54,...,0,42,11,41,12,23,30,51,35,32
3,Yceag6Ca0B.jpg,12,32,52,5,15,42,55,4,10,...,34,20,13,21,22,45,33,41,23,35
4,nDBcp5ZyPj.jpg,33,44,12,45,50,54,20,24,0,...,55,34,4,25,52,15,11,21,2,32


In [4]:
columns = df1.columns.tolist()
for i in columns[1:]:
    df1[i] = df1[i].replace('0', '00')
    df1[i] = df1[i].replace('1', '01')
    df1[i] = df1[i].replace('2', '02')
    df1[i] = df1[i].replace('3', '03')
    df1[i] = df1[i].replace('4', '04')
    df1[i] = df1[i].replace('5', '05')

df1.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,hwJUKjziH0.jpg,22,20,14,04,43,40,35,53,25,...,31,41,13,32,33,10,02,30,50,44
1,smzyIVQ4Ve.jpg,03,50,43,14,05,13,33,52,01,...,41,35,45,30,44,22,51,00,32,15
2,IT3H2QbaJY.jpg,55,25,43,10,33,20,22,13,54,...,00,42,11,41,12,23,30,51,35,32
3,Yceag6Ca0B.jpg,12,32,52,05,15,42,55,04,10,...,34,20,13,21,22,45,33,41,23,35
4,nDBcp5ZyPj.jpg,33,44,12,45,50,54,20,24,00,...,55,34,04,25,52,15,11,21,02,32


val, train split on the dataframes

In [5]:
n1 = len(df1)
n2 = len(df2)
val_split = 0.2
val_df1 = df1[:int(n1*val_split)]
train_df1 = df1[int(n1*val_split):]

val_df2 = df2[:int(n2*0.2)]
train_df2 = df2[int(n2*0.2):]

In [6]:
print(train_df1.shape)

(2456, 37)


partitions image to 36*36 tiles

In [7]:
def load_images1(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\faces" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

def load_images2(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\landmarks" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)
# newly_added
def load_images(new_images):
    ret = []
    for i in range(len(new_images)):
        pieces = []
        img = new_images[i]
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

In [8]:
train_img1 = load_images1(train_df1)
val_img1 = load_images1(val_df1)

train_img2 = load_images2(train_df2)
val_img2 = load_images2(val_df2)


100%|██████████| 600/600 [00:01<00:00, 583.40it/s]


In [9]:
"""train_img1 = train_img1/255
train_img2 = train_img2/255
val_img1 = val_img1/255
val_img2 = val_img2/255"""

'train_img1 = train_img1/255\ntrain_img2 = train_img2/255\nval_img1 = val_img1/255\nval_img2 = val_img2/255'

Generates labels to each tile of image 

In [10]:
def load_labels(df):
    ret = []
    for idx in tqdm(df.index):
        label = np.zeros(shape=(36,36))
        for i in range(6):
            for j in range(6):
                place = i*6 + j
                val = int(df.loc[idx, str(i)+str(j)][0])*6 + int(df.loc[idx, str(i)+str(j)][1])
                label[place][val] = 1
        ret.append(label)
    return np.array(ret)


train_labels1 = load_labels(train_df1)
val_labels1 = load_labels(val_df1)

train_labels2 = load_labels(train_df2)
val_labels2 = load_labels(val_df2)



100%|██████████| 600/600 [00:00<00:00, 1410.64it/s]


In [11]:
print(train_img1.shape)

(2456, 36, 50, 50, 3)


In [12]:
print(train_labels1[0].shape)

(36, 36)


model description

In [13]:

inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

#here we only define the layers and feed the input later
conv1 = tf.keras.layers.Conv2D(filters = 10,kernel_size= (5,5), strides=5, padding='same')
#max_pooling1=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv2 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')
#max_pooling2=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv3 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')
flat = tf.keras.layers.Flatten()

drop = tf.keras.layers.Dropout(.5)

#convLSTM = tf.keras.layers.ConvLSTM2D(10, (5,5), padding='same',  activation='tanh', recurrent_activation='hard_sigmoid', input_shape=(50,50,3))
dense1 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))

dense2 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))

#now we feed each puzzle piece one by one to the layers and store the encodings
mid = []
for i in range(36):
    x = conv1(inp[:,i]) #first dimension is batch_size
    #x = max_pooling1(x)
    x = conv2(x)
    x = conv3(x)
    
    #x = max_pooling2(x)
    x = flat(x)
    
    mid.append(x)
print(x.shape)
x = tf.keras.layers.concatenate(mid, axis=-1)
print(x.shape)
x = dense1(x)
drop
x = dense2(x)
print(x.shape)
x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

(None, 10)
(None, 360)
(None, 1296)


In [14]:
"""inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

#here we only define the layers and feed the input later
conv1 = tf.keras.layers.Conv2D(filters = 10,kernel_size= (5,5), strides=1, padding='same')
#max_pooling1=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv2 = tf.keras.layers.Conv2D(10, (5,5), strides=1, padding='same')
#max_pooling2=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv3 = tf.keras.layers.Conv2D(100, (5,5), strides=5, padding='same')

conv4 = tf.keras.layers.Conv2D(100, (5,5), 5, padding='same')

conv5 = tf.keras.layers.Conv2D(100, (5,5), 5, padding='same')
flat = tf.keras.layers.Flatten()

drop = tf.keras.layers.Dropout(.5)

#convLSTM = tf.keras.layers.ConvLSTM2D(10, (5,5), padding='same',  activation='tanh', recurrent_activation='hard_sigmoid', input_shape=(50,50,3))
dense1 = tf.keras.layers.Dense(36, kernel_regularizer=tf.keras.regularizers.l2(0.001))

dense2 = tf.keras.layers.Dense(36, kernel_regularizer=tf.keras.regularizers.l2(0.001))

dense3 = tf.keras.layers.Dense(36, kernel_regularizer=tf.keras.regularizers.l2(0.001))

#now we feed each puzzle piece one by one to the layers and store the encodings
mid = []
for i in range(36):
    x = conv1(inp[:,i]) #first dimension is batch_size
    #x = max_pooling1(x)
    x = conv2(x)
    #x = conv3(x)
    #x = conv4(x)
    #x = conv5(x)
    #x = max_pooling2(x)
    x = flat(x)
    x = dense1(x)
    drop
    x = dense2(x)
    #drop
    #x = dense3(x)
    mid.append(x)

#x = tf.keras.layers.concatenate(mid, axis=-1)
#x = dense1(x)
#drop
#x = dense2(x)
print(x.shape)
x = tf.keras.layers.concatenate(mid, axis=-1)
print(x.shape)
x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability"""

(None, 36)
(None, 1296)


In [14]:
model1 = tf.keras.Model(inputs=inp, outputs=x)
model1.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=["accuracy"])

model2 = tf.keras.Model(inputs=inp, outputs=x)
model2.compile(optimizer="RMSprop", loss="categorical_crossentropy")


In [16]:
print((mid))

[<KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTenso

In [17]:
from keras.callbacks import ModelCheckpoint
model_cp = ModelCheckpoint(filepath="best.h5", monitor="val_accuracy", verbose=1)

model trained over faces

In [16]:
model1.fit(x=train_img1, 
        y=train_labels1, 
        batch_size=16, 
        validation_data=[val_img1, val_labels1], 
        shuffle=True,
        epochs = 15)

Epoch 1/15
154/154 [==============================] - 14s 88ms/step - loss: 2.9491 - accuracy: 0.3658 - val_loss: 4.9635 - val_accuracy: 0.0576
Epoch 2/15
154/154 [==============================] - 12s 79ms/step - loss: 2.8988 - accuracy: 0.3790 - val_loss: 5.0540 - val_accuracy: 0.0574
Epoch 3/15
154/154 [==============================] - 12s 79ms/step - loss: 2.8539 - accuracy: 0.3921 - val_loss: 4.9732 - val_accuracy: 0.0621
Epoch 4/15
154/154 [==============================] - 12s 80ms/step - loss: 2.8214 - accuracy: 0.4025 - val_loss: 5.0212 - val_accuracy: 0.0624
Epoch 5/15
154/154 [==============================] - 12s 80ms/step - loss: 2.7981 - accuracy: 0.4087 - val_loss: 5.0730 - val_accuracy: 0.0639
Epoch 6/15
154/154 [==============================] - 12s 80ms/step - loss: 2.7834 - accuracy: 0.4146 - val_loss: 5.1744 - val_accuracy: 0.0641
Epoch 7/15
154/154 [==============================] - 13s 82ms/step - loss: 2.7611 - accuracy: 0.4219 - val_loss: 5.3540 - val_accuracy:

model trained over landmarks

In [17]:
model2.fit(x=train_img2, 
        y=train_labels2, 
        batch_size=16, 
        validation_data=[val_img2, val_labels2], 
        shuffle=True,
        epochs=5)

Epoch 1/5
150/150 [==============================] - 18s 94ms/step - loss: 4.2675 - val_loss: 3.8794
Epoch 2/5
150/150 [==============================] - 12s 82ms/step - loss: 3.6727 - val_loss: 3.9485
Epoch 3/5
150/150 [==============================] - 12s 78ms/step - loss: 3.4150 - val_loss: 4.1275
Epoch 4/5
150/150 [==============================] - 12s 77ms/step - loss: 3.2503 - val_loss: 4.3446
Epoch 5/5
150/150 [==============================] - 12s 78ms/step - loss: 3.1561 - val_loss: 4.4808


In [18]:
faces_lis = os.listdir(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\faces")
landmarks_lis = os.listdir(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\landmarks")

In [19]:
df4 = pd.DataFrame()
df4.insert(0, "image", faces_lis)

df5 = pd.DataFrame()
df5.insert(0, "image", landmarks_lis)

In [20]:
def load_images_facest(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\faces" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

def load_images_landmarkst(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\landmarks" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

In [21]:
img_facest = load_images_facest(df4)
img_landmarkst = load_images_landmarkst(df5)

100%|██████████| 1096/1096 [00:02<00:00, 474.79it/s]


skip

In [22]:
pred1 = model1.predict(val_img1)
pred1 = np.argmax(pred1, axis=-1)

pred2 = model2.predict(val_img2)
pred2 = np.argmax(pred2, axis=-1)

19/19 [==============================] - 2s 46ms/step


In [23]:
pred4 = model1.predict(img_facest)
pred4 = np.argmax(pred4, axis=-1)

pred5 = model2.predict(img_landmarkst)
pred5 = np.argmax(pred5, axis=-1)

35/35 [==============================] - 2s 51ms/step


skip

skip


In [24]:
pred_list1 = []
for i in range(pred1.shape[0]):
    t = []
    for j in range(pred1[i].shape[0]):
        t.append(str(pred1[i][j]//6) + str(pred1[i][j]%6))
    pred_list1.append(t)

image_df1 = pd.DataFrame(val_df1["image"])
label_df1 = pd.DataFrame(pred_list1)
out1 = pd.concat([image_df1, label_df1], axis=1)
out1.columns = df1.columns

skip

In [25]:
pred_list2 = []
for i in range(pred2.shape[0]):
    t = []
    for j in range(pred2[i].shape[0]):
        t.append(str(pred2[i][j]//6) + str(pred2[i][j]%6))
    pred_list2.append(t)

image_df2 = pd.DataFrame(val_df2["image"])
label_df2 = pd.DataFrame(pred_list2)
out2 = pd.concat([image_df2, label_df2], axis=1)
out2.columns = df2.columns

In [26]:
pred_list3 = []
for i in range(pred4.shape[0]):
    t = []
    for j in range(pred4[i].shape[0]):
        t.append(str(pred4[i][j]//6) + str(pred4[i][j]%6))
    pred_list3.append(t)

image_df3 = pd.DataFrame(df4["image"])
label_df3 = pd.DataFrame(pred_list3)
out3 = pd.concat([image_df3, label_df3], axis=1)
out3.columns = df1.columns

pred_list4 = []
for i in range(pred5.shape[0]):
    t = []
    for j in range(pred5[i].shape[0]):
        t.append(str(pred5[i][j]//6) + str(pred5[i][j]%6))
    pred_list4.append(t)

image_df4 = pd.DataFrame(df5["image"])
label_df4 = pd.DataFrame(pred_list4)
out4 = pd.concat([image_df4, label_df4], axis=1)
out4.columns = df2.columns

In [27]:
out3.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,00iOHCZTws.jpg,02,43,30,41,31,50,51,52,04,...,41,53,34,05,00,20,12,11,44,32
1,03fxX7jYxI.jpg,31,40,30,54,05,21,03,20,05,...,04,04,33,31,13,44,30,04,05,54
2,05evYM76RD.jpg,00,55,21,31,25,01,23,05,55,...,21,50,01,01,33,50,12,20,52,35
3,08UaxmNJtO.jpg,45,44,02,55,05,11,35,55,33,...,33,01,15,45,15,54,41,42,21,20
4,09gb5hcslV.jpg,04,12,10,45,52,12,53,34,55,...,44,50,31,22,13,25,43,52,52,25


In [28]:
def prediction_to_submission(df):
    df = df.copy()
    for s in ['r','c']:
        for i in range(6):
            for j in range(6):
                df[str(s)+str(i)+str(j)] = None

    for idx in df.index:
        for i in range(6):
            for j in range(6):
                df.loc[idx, 'r'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][0])
                df.loc[idx, 'c'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][1])

    columns_to_remove = []
    for i in range(6):
        for j in range(6):
            columns_to_remove.append(str(i)+str(j))

    return df.drop(columns=columns_to_remove)
submission1 = prediction_to_submission(out3)
submission2 = prediction_to_submission(out4)

submission = pd.concat([submission1, submission2])
submission.head()

,image,r00,r01,r02,r03,r04,r05,r10,r11,r12,...,c42,c43,c44,c45,c50,c51,c52,c53,c54,c55
0,00iOHCZTws.jpg,0,4,3,4,3,5,5,5,0,...,1,3,4,5,0,0,2,1,4,2
1,03fxX7jYxI.jpg,3,4,3,5,0,2,0,2,0,...,4,4,3,1,3,4,0,4,5,4
2,05evYM76RD.jpg,0,5,2,3,2,0,2,0,5,...,1,0,1,1,3,0,2,0,2,5
3,08UaxmNJtO.jpg,4,4,0,5,0,1,3,5,3,...,3,1,5,5,5,4,1,2,1,0
4,09gb5hcslV.jpg,0,1,1,4,5,1,5,3,5,...,4,0,1,2,3,5,3,2,2,5


In [43]:
lis = ['1','3','4','5']
lis.remove('1')
print(lis)

['3', '4', '5']


In [45]:
com = []
for i in range(6):
    com.append(str(i))
df9 = deepcopy(submission)
for i in range(df9.shape[0]):
    com = []
    for x in range(6):
        com.append(str(x))
    for j in range(df9.shape[1]):
        y = random.choice(com)
        df9.iloc[i][j] = y
        #com.remove(y)
df9.head()


,image,r00,r01,r02,r03,r04,r05,r10,r11,r12,...,c42,c43,c44,c45,c50,c51,c52,c53,c54,c55
0,1,3,3,3,1,3,4,4,0,4,...,4,0,5,0,4,5,2,5,4,4
1,3,3,5,5,5,5,2,0,4,5,...,2,0,4,4,1,4,3,2,5,4
2,4,2,3,4,3,0,1,3,3,5,...,4,3,3,3,3,5,3,5,4,4
3,5,3,4,4,2,1,3,1,0,0,...,1,2,2,4,2,2,4,5,2,5
4,0,3,1,0,2,0,0,4,2,5,...,4,5,4,0,3,4,5,1,5,1


In [29]:
print(df9.shape)

(1996, 73)


In [48]:
df9.to_csv("opt1.csv", index = False)